In [79]:
import pandas as pd


df_sino_neg = pd.read_csv("./data/clustered/cinovac_neg.csv", sep=";")
df_sino_pos = pd.read_csv("./data/clustered/cinovac_pos.csv", sep=";")
df_cloroquina_neg = pd.read_csv("./data/clustered/cloroquina_neg.csv", sep=";")
df_cloroquina_pos = pd.read_csv("./data/clustered/cloroquina_pos.csv", sep=";")
df_igreja_neg = pd.read_csv("./data/clustered/igreja_neg.csv", sep=";")
df_igreja_pos = pd.read_csv("./data/clustered/igreja_pos.csv", sep=";")

In [80]:
print(df_sino_neg[df_sino_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)
print(df_sino_pos[df_sino_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)

print(df_cloroquina_neg[df_cloroquina_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)
print(df_cloroquina_pos[df_cloroquina_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)

print(df_igreja_neg[df_igreja_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)
print(df_igreja_pos[df_igreja_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1].shape)

df_sino_neg = df_sino_neg[df_sino_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]
df_sino_pos = df_sino_pos[df_sino_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]

df_cloroquina_neg = df_cloroquina_neg[df_cloroquina_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]
df_cloroquina_pos = df_cloroquina_pos[df_cloroquina_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]

df_igreja_neg = df_igreja_neg[df_igreja_neg["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]
df_igreja_pos = df_igreja_pos[df_igreja_pos["Voce considera que o segundo texto é uma reecrita do primeiro?1 - Sim2 - Não "] == 1]


(4, 8)
(4, 8)
(3, 8)
(8, 8)
(0, 8)
(6, 8)


In [81]:
from nltk import ngrams
import numpy as np
def tokenize(text, nGram=3):
    text = text.split()
    grams = []
    for i in range(1, nGram):
        i_grams = [
            " ".join(gram)
            for gram in ngrams(text, i)
        ]
        grams.extend(i_grams)
        
    return grams

def getNgramOverlap(hypothesys, references, nGram):

  overlaps = []
  for h, r in zip(hypothesys, references):
    if (h.strip() == "") or (r.strip() == ""):
      overlaps.append(1.0)
      continue
    a = tokenize(h, nGram)
    b = tokenize(r, nGram)
    if len(a) >= len(b):
      overlaps.append(len(set(a) & set(b))/len(a))
    elif len(b) >= len(a):
      overlaps.append(len(set(a) & set(b))/len(b))

  return overlaps


def taxa_sobreposicao_pronomes(lista1, lista2):
    set1 = set(lista1)
    set2 = set(lista2)

    # Calcula a interseção e a união dos conjuntos
    intersecao = set1.intersection(set2)
    uniao = set1.union(set2)

    # Calcula a taxa de sobreposição (Jaccard)
    if len(uniao) == 0:
        return 0.0
    else:
        return len(intersecao) / len(uniao)
    
import spacy
from tqdm import tqdm
import statistics
tqdm.pandas()
nlp = spacy.load("pt_core_news_lg")
def get_pos(x):
    pos = []
    for s in x:
        s_pos = []
        for w in nlp(s):
            s_pos.append(w.pos_)
        pos.append(s_pos)
    return pos


def get_avg_std(lista, busca):
    ocorrencias = []
    
    for sentenca in lista:
        ocorrencias.append(sentenca.count(busca))
    if len(ocorrencias) == 0:
        return 0.0, 0.0
    else:
        return statistics.mean(ocorrencias), statistics.stdev(ocorrencias)


In [103]:
"""
Calcula a taxa de sobreposição de pronomes entre duas sentenças
"""
import ast
taxas = []

for line in df_sino_neg.iterrows():

    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(np.mean(taxas))

taxas = []

for line in df_sino_pos.iterrows():

    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(np.mean(taxas))

print("=====================================")

taxas = []

for line in df_cloroquina_neg.iterrows():

    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(np.mean(taxas))

taxas = []

for line in df_cloroquina_pos.iterrows():
    
    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(np.mean(taxas))

print("=====================================")

taxas = []

for line in df_igreja_neg.iterrows():
    
    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(0.0)

taxas = []

for line in df_igreja_pos.iterrows():
    
    taxas.append(taxa_sobreposicao_pronomes(ast.literal_eval(line[1]["nouns_origem"]).keys(), ast.literal_eval(line[1]["nouns_alvo"]).keys()))

print(np.mean(taxas))




    


0.11970551378446115
0.08173076923076923
0.12314814814814816
0.04834620716973659
0.0
0.19259259259259257


In [82]:
print(np.mean(getNgramOverlap(df_sino_neg["textos_origem"], df_sino_neg["textos_alvo"], nGram=3)))
print(np.mean(getNgramOverlap(df_sino_pos["textos_origem"], df_sino_pos["textos_alvo"], nGram=3)))

print("========================================")

print(np.mean(getNgramOverlap(df_cloroquina_neg["textos_origem"], df_cloroquina_neg["textos_alvo"], nGram=3)))
print(np.mean(getNgramOverlap(df_cloroquina_pos["textos_origem"], df_cloroquina_pos["textos_alvo"], nGram=3)))

print("========================================")

#print(np.mean(getNgramOverlap(df_igreja_neg["textos_origem"], df_igreja_neg["textos_alvo"], nGram=3))) 0.0 0.0
print(0.0)
print(np.mean(getNgramOverlap(df_igreja_pos["textos_origem"], df_igreja_pos["textos_alvo"], nGram=3)))


0.05998490202728576
0.08769113941527734
0.02899601304820587
0.045931740501886476
0.0
0.043877210543877214


In [83]:
  
pos_origem_igreja = get_pos(df_igreja_pos["textos_origem"].to_list()+df_igreja_pos["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(pos_origem_igreja, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(pos_origem_igreja, "PUNCT"))
print("avg. and std. ADP", get_avg_std(pos_origem_igreja, "ADP"))
print("avg. and std. VERB", get_avg_std(pos_origem_igreja, "VERB"))
print("avg. and std. DET", get_avg_std(pos_origem_igreja, "DET"))
print("avg. and std. PRON", get_avg_std(pos_origem_igreja, "PRON"))
print("avg. and std. ADV", get_avg_std(pos_origem_igreja, "ADV"))
print("avg. and std. PROPN", get_avg_std(pos_origem_igreja, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(pos_origem_igreja, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(pos_origem_igreja, "ADJ"))

print("=====================================")

neg_origem_igreja = get_pos(df_igreja_neg["textos_origem"].to_list()+df_igreja_neg["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(neg_origem_igreja, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(neg_origem_igreja, "PUNCT"))
print("avg. and std. ADP", get_avg_std(neg_origem_igreja, "ADP"))
print("avg. and std. VERB", get_avg_std(neg_origem_igreja, "VERB"))
print("avg. and std. DET", get_avg_std(neg_origem_igreja, "DET"))
print("avg. and std. PRON", get_avg_std(neg_origem_igreja, "PRON"))
print("avg. and std. ADV", get_avg_std(neg_origem_igreja, "ADV"))
print("avg. and std. PROPN", get_avg_std(neg_origem_igreja, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(neg_origem_igreja, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(neg_origem_igreja, "ADJ"))

avg. and std. NOUN (3, 1.4770978917519928)
avg. and std. PUNCT (0.3333333333333333, 0.49236596391733095)
avg. and std. ADP (1.6666666666666667, 1.1547005383792515)
avg. and std. VERB (2.5, 1.0)
avg. and std. DET (0.5, 0.5222329678670935)
avg. and std. PRON (1.1666666666666667, 0.7177405625652734)
avg. and std. ADV (0.8333333333333334, 0.7177405625652734)
avg. and std. PROPN (0.16666666666666666, 0.3892494720807615)
avg. and std. CCONJ (0.5, 0.5222329678670935)
avg. and std. ADJ (1, 0.6030226891555273)
avg. and std. NOUN (0.0, 0.0)
avg. and std. PUNCT (0.0, 0.0)
avg. and std. ADP (0.0, 0.0)
avg. and std. VERB (0.0, 0.0)
avg. and std. DET (0.0, 0.0)
avg. and std. PRON (0.0, 0.0)
avg. and std. ADV (0.0, 0.0)
avg. and std. PROPN (0.0, 0.0)
avg. and std. CCONJ (0.0, 0.0)
avg. and std. ADJ (0.0, 0.0)


In [84]:
pos_origem_sino = get_pos(df_sino_pos["textos_origem"].to_list()+df_sino_pos["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(pos_origem_sino, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(pos_origem_sino, "PUNCT"))
print("avg. and std. ADP", get_avg_std(pos_origem_sino, "ADP"))
print("avg. and std. VERB", get_avg_std(pos_origem_sino, "VERB"))
print("avg. and std. DET", get_avg_std(pos_origem_sino, "DET"))
print("avg. and std. PRON", get_avg_std(pos_origem_sino, "PRON"))
print("avg. and std. ADV", get_avg_std(pos_origem_sino, "ADV"))
print("avg. and std. PROPN", get_avg_std(pos_origem_sino, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(pos_origem_sino, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(pos_origem_sino, "ADJ"))

print("=====================================")
neg_origem_sino = get_pos(df_sino_neg["textos_origem"].to_list()+df_sino_neg["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(neg_origem_sino, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(neg_origem_sino, "PUNCT"))
print("avg. and std. ADP", get_avg_std(neg_origem_sino, "ADP"))
print("avg. and std. VERB", get_avg_std(neg_origem_sino, "VERB"))
print("avg. and std. DET", get_avg_std(neg_origem_sino, "DET"))
print("avg. and std. PRON", get_avg_std(neg_origem_sino, "PRON"))
print("avg. and std. ADV", get_avg_std(neg_origem_sino, "ADV"))
print("avg. and std. PROPN", get_avg_std(neg_origem_sino, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(neg_origem_sino, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(neg_origem_sino, "ADJ"))


avg. and std. NOUN (4.125, 3.758324094593227)
avg. and std. PUNCT (1.375, 1.0606601717798212)
avg. and std. ADP (3.5, 1.6903085094570331)
avg. and std. VERB (2.5, 1.0690449676496976)
avg. and std. DET (2.75, 1.3887301496588271)
avg. and std. PRON (1.125, 0.8345229603962802)
avg. and std. ADV (0.875, 0.6408699444616558)
avg. and std. PROPN (2.375, 1.1877349391654208)
avg. and std. CCONJ (0.375, 0.7440238091428449)
avg. and std. ADJ (1.25, 1.5811388300841898)
avg. and std. NOUN (4.75, 2.9640705601780613)
avg. and std. PUNCT (3.875, 1.807721533549109)
avg. and std. ADP (4.625, 1.9226098333849673)
avg. and std. VERB (5.25, 1.8322507626258087)
avg. and std. DET (2.75, 1.0350983390135313)
avg. and std. PRON (1.25, 0.8864052604279183)
avg. and std. ADV (2.875, 1.552647508520297)
avg. and std. PROPN (3, 2.563479777846623)
avg. and std. CCONJ (1, 0.9258200997725514)
avg. and std. ADJ (2, 1.4142135623730951)


In [85]:
pos_origem_cloroquina = get_pos(df_cloroquina_pos["textos_origem"].to_list()+df_cloroquina_pos["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(pos_origem_cloroquina, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(pos_origem_cloroquina, "PUNCT"))
print("avg. and std. ADP", get_avg_std(pos_origem_cloroquina, "ADP"))
print("avg. and std. VERB", get_avg_std(pos_origem_cloroquina, "VERB"))
print("avg. and std. DET", get_avg_std(pos_origem_cloroquina, "DET"))
print("avg. and std. PRON", get_avg_std(pos_origem_cloroquina, "PRON"))
print("avg. and std. ADV", get_avg_std(pos_origem_cloroquina, "ADV"))
print("avg. and std. PROPN", get_avg_std(pos_origem_cloroquina, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(pos_origem_cloroquina, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(pos_origem_cloroquina, "ADJ"))


print("=====================================")

neg_origem_cloroquina = get_pos(df_cloroquina_neg["textos_origem"].to_list()+df_cloroquina_neg["textos_alvo"].to_list())

print("avg. and std. NOUN", get_avg_std(neg_origem_cloroquina, "NOUN"))
print("avg. and std. PUNCT", get_avg_std(neg_origem_cloroquina, "PUNCT"))
print("avg. and std. ADP", get_avg_std(neg_origem_cloroquina, "ADP"))
print("avg. and std. VERB", get_avg_std(neg_origem_cloroquina, "VERB"))
print("avg. and std. DET", get_avg_std(neg_origem_cloroquina, "DET"))
print("avg. and std. PRON", get_avg_std(neg_origem_cloroquina, "PRON"))
print("avg. and std. ADV", get_avg_std(neg_origem_cloroquina, "ADV"))
print("avg. and std. PROPN", get_avg_std(neg_origem_cloroquina, "PROPN"))
print("avg. and std. CCONJ", get_avg_std(neg_origem_cloroquina, "CCONJ"))
print("avg. and std. ADJ", get_avg_std(neg_origem_cloroquina, "ADJ"))



avg. and std. NOUN (6.5625, 3.982775414205526)
avg. and std. PUNCT (4.8125, 2.372586492979058)
avg. and std. ADP (3.625, 1.857417562100671)
avg. and std. VERB (5.4375, 3.0977142110487423)
avg. and std. DET (3.875, 3.263433774416144)
avg. and std. PRON (1.8125, 1.4244882121894398)
avg. and std. ADV (2.3125, 2.2721135535003527)
avg. and std. PROPN (2.9375, 3.2958306995353994)
avg. and std. CCONJ (1.4375, 0.963932916061417)
avg. and std. ADJ (1.125, 1.1474609652039003)
avg. and std. NOUN (4.333333333333333, 3.32665998663324)
avg. and std. PUNCT (1.8333333333333333, 1.7224014243685084)
avg. and std. ADP (3.3333333333333335, 3.444802848737017)
avg. and std. VERB (3.3333333333333335, 1.2110601416389966)
avg. and std. DET (1.3333333333333333, 1.632993161855452)
avg. and std. PRON (2, 2.0)
avg. and std. ADV (0.6666666666666666, 0.5163977794943223)
avg. and std. PROPN (0.8333333333333334, 1.602081978759722)
avg. and std. CCONJ (0.8333333333333334, 1.1690451944500122)
avg. and std. ADJ (1.5, 1.7

In [106]:
from scipy.stats import kendalltau
from sklearn.feature_extraction.text import CountVectorizer

# Função que calcula a correlação de Kendall (tau) entre duas listas de textos
def correlacao_tau_textos(lista1, lista2):
    # Combina as duas listas
    textos_completos = lista1 + lista2

    # Converte os textos para representações vetoriais (bag-of-words)
    vectorizer = CountVectorizer()
    matriz_vetores = vectorizer.fit_transform(textos_completos).toarray()

    # Divide novamente as matrizes
    vetores_lista1 = matriz_vetores[:len(lista1)].sum(axis=1)
    vetores_lista2 = matriz_vetores[len(lista1):].sum(axis=1)

    # Calcula a correlação tau de Kendall
    tau, p_valor = kendalltau(vetores_lista1, vetores_lista2)

    return tau, p_valor



In [109]:
# Calcula a correlação de tau entre os textos

lista1 = df_cloroquina_neg["textos_origem"].to_list()
lista2 = df_cloroquina_neg["textos_alvo"].to_list()

tau, p_valor = correlacao_tau_textos(lista1, lista2)
print(f"Correlação tau: {tau}\nP-valor: {p_valor}")

lista1 = df_cloroquina_pos["textos_origem"].to_list()
lista2 = df_cloroquina_pos["textos_alvo"].to_list()

tau, p_valor = correlacao_tau_textos(lista1, lista2)

print(f"Correlação tau: {tau}\nP-valor: {p_valor}")


print("=====================================")

lista1 = df_sino_neg["textos_origem"].to_list()
lista2 = df_sino_neg["textos_alvo"].to_list()

tau, p_valor = correlacao_tau_textos(lista1, lista2)

print(f"Correlação tau: {tau}\nP-valor: {p_valor}")

lista1 = df_sino_pos["textos_origem"].to_list()
lista2 = df_sino_pos["textos_alvo"].to_list()

tau, p_valor = correlacao_tau_textos(lista1, lista2)

print(f"Correlação tau: {tau}\nP-valor: {p_valor}")

print("=====================================")

lista1 = df_igreja_neg["textos_origem"].to_list()
lista2 = df_igreja_neg["textos_alvo"].to_list()

tau, p_valor = 0.0 , 0.0

print(f"Correlação tau: {tau}\nP-valor: {p_valor}")

lista1 = df_igreja_pos["textos_origem"].to_list()
lista2 = df_igreja_pos["textos_alvo"].to_list()

tau, p_valor = correlacao_tau_textos(lista1, lista2)

print(f"Correlação tau: {tau}\nP-valor: {p_valor}")




Correlação tau: 0.816496580927726
P-valor: 0.22067136191984693
Correlação tau: 0.32732683535398854
P-valor: 0.2618277009271762
Correlação tau: 0.0
P-valor: 1.0
Correlação tau: 0.7745966692414834
P-valor: 0.15729920705028502
Correlação tau: 0.0
P-valor: 0.0
Correlação tau: 0.07692307692307693
P-valor: 0.8402142310487369
